## Setting

In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib.pyplot as plt   # plots
import numpy as np
import time
import warnings

from liblibra_core import *
import util.libutil as comn
from libra_py import units
import libra_py.models.Holstein as Holstein
import libra_py.models.Tully as Tully
import libra_py.models.Subotnik as Subotnik
import libra_py.models.Esch_Levine as Esch_Levine
from libra_py import dynamics_plotting
import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot
import libra_py.data_savers as data_savers

import libra_py.data_read as data_read

#from recipes import shxf, mqcxf, ehxf
#from recipes import ehrenfest_adi_ld, ehrenfest_adi_nac, ehrenfest_dia, mfsd
#from recipes import fssh, sdm, bcsh

import libra_py.dynamics.exact.compute as dvr
import libra_py.dynamics.exact.save as dvr_save

#from matplotlib.mlab import griddata
#%matplotlib inline 
warnings.filterwarnings('ignore')

colors = {}
colors.update({"11": "#D20C0C", "12": "#FF5339", "13": "#FF8965"}) #red
colors.update({"21": "#00790C", "22": "#49A93F", "23": "#7DDB6D"}) #green
colors.update({"31": "#0009FF", "32": "#7145FF", "33": "#B176FF"}) #blue
colors.update({"41": "#FFA100", "42": "#FFD249", "43": "#FFFF7C"}) #orange
colors.update({"51": "#FF1DCE", "52": "#FFCDFF"}) #magenta
colors.update({"61": "#00ffea", "62": "#70FFFF", "63": "#AFFFFF"}) #cyan
colors.update({"71": "#bd20ff", "72": "#F461FF", "73": "#FF97FF"}) #indigo

clrs_index = ["11", "21", "31", "41", "51", "61", "71", "12", "22", "32", "42", "52", "62", "72" "13","23", "33", "53", "73"]

In [ ]:
def compute_model(q, params, full_id):

    model = params["model"]
    res = None
    
    if model==1:        
        res = Holstein.Holstein2(q, params, full_id) 
    elif model==2:
        res = Tully.Tully3(q, params, full_id)
    elif model==3:
        res = Subotnik.double_arch_geometry(q, params, full_id)
    elif model==4:
        res = Esch_Levine.JCP_2020(q, params, full_id)
    else:
        pass            

    return res

In [ ]:
def potential(q, params):
    full_id = Py2Cpp_int([0,0]) 
    
    return compute_model(q, params, full_id)

In [ ]:
model_params1 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.000}
model_params2 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.001}
#model_params3 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.01}
model_params3 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0,  0.0], "x_n":[0.0,  2.5],"k_n":[0.002, 0.005],"V":0.05}
model_params4 = {"model":1, "model0":1, "nstates":2, "E_n":[0.0, -0.01], "x_n":[0.0,  0.5],"k_n":[0.002, 0.008],"V":0.001}

model_params5 = {"model":2, "model0":2, "nstates":2} # ECR
model_params6 = {"model":3, "model0":3, "nstates":2} # DAG

model_params7 = {"model":4, "model0":4, "nstates":2, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.0, "i_crit":2, "delta":0.01 } # Esch-Levine

model_params8 = {"model":4, "model0":4, "nstates":3, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.0, "i_crit":3, "delta":0.01 } # Esch-Levine

model_params9 = {"model":4, "model0":4, "nstates":5, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.0, "i_crit":4, "delta":0.01 } # Esch-Levine

model_params10 = {"model":4, "model0":4, "nstates":5, 
                 "w0":0.015, "w1":0.005, "V":0.005, "eps":0.02, "i_crit":3, "delta":0.01 } # Esch-Levine

all_model_params = [model_params1, model_params2, model_params3, model_params4, 
                    model_params5, model_params6, 
                    model_params7, model_params8, model_params9, model_params10
                   ]

In [ ]:
# 0 - Holstein, trivial crossing, 2 level
# 1 - Holstein, strong nonadiabatic, 2 level
# 2 - Holstein, adiabatic, 2 level
# 3 - Holstein, double crossing, strong nonadiabatic, 2 level
# 4 - Tully, extended crossing with reflection, 2 level
# 5 - Double arch geometry or symmetrized ECWR, 2 level
# 6 - Esch-Levine, LZ-like, 2 level
# 7 - Esch-Levine, 1 crosses 2 parallel, 3 level
# 8 - Esch-Levine, 1 crosses 4 evenly-spaced parallel, 5 level
# 9 - Esch-Levine, 1 crosses 4 parallel split into 2 groups, 5 level

#################################
# Give the model used an index
model_indx = 7
################################

model_params = all_model_params[model_indx]

In [ ]:
#################################
# Give the method index
method_indx = 7
################################

## Compute PEC, td energy

In [ ]:
#Setting
freq = 40*10
#freq = 10*10

ntraj = 2000

if model_params["model"] == 1: #Holstein
    xmin = -10.0
    xmax = 10.0
    nx = 1000
    dx = (xmax - xmin)/nx
    xs = [xmin + dx*i for i in range(nx)]

    ymin = -0.03
    ymax = 0.1
    
    nsteps = 8000
    
elif model_params["model"] == 2 : #ECR
    xmin = -30.0
    xmax = 30.0
    nx = 1000
    dx = (xmax - xmin)/nx
    xs = [xmin + dx*i for i in range(nx)]

    ymin = -0.4
    ymax = 0.4
elif model_params["model"] == 3: #DAG
    xmin = -30.0
    xmax = 30.0
    nx = 1000
    dx = (xmax - xmin)/nx
    xs = [xmin + dx*i for i in range(nx)]

    ymin = -0.2
    ymax = 0.2
elif model_params["model"] == 4: #Esch-Levine
    xmin = -20.0
    xmax = 20.0
    nx = 1000
    dx = (xmax - xmin)/nx
    xs = [xmin + dx*i for i in range(nx)]

    ymin = -0.1
    ymax = 0.1
    
    nsteps = 25000

# Adiabatic surface information
def get_Eadi(nst, q):
    res = potential(q, model_params)
    H = np.zeros((nst, nst), dtype=complex)
    for i in range(nst):
        for j in range(nst):
            H[i,j] = res.ham_dia.get(i,j)
    E, U = np.linalg.eig(H) 
    idx= np.argsort(E) 
    E = E[idx] 
    U = U[:,idx]

    return E

Es = []

nst = model_params["nstates"]
q = MATRIX(1,1) # temporary for a nuclear coordinate
for ix in range(nx):
    val = xmin + ix*dx
    q.set(0, 0, val)
    E = get_Eadi(nst, q)

    Es.append(E)

Es = np.array(Es)

fn = F"model{model_indx}-method{method_indx}-icond0/mem_data.hdf"
    
with h5py.File(fn, 'r') as f:
#with h5py.File(F"model{model_indx}-method{method_indx}-icond0/mem_data.hdf", 'r') as f:
    TS = f["time/data"][:nsteps:freq]
    QS = f["q/data"][:nsteps:freq, :ntraj, 0]
    STATES = f["states/data"][:nsteps:freq,:ntraj]
    C_ADI = f["Cadi/data"][:nsteps:freq,:ntraj,:]

#with h5py.File(F"exact-model{model_indx}-icond{icond_indx}/data.hdf", 'r') as f:
#    PSI_ADI = f["PSI_adi/data"][:,:,:]
#RGRID = [exact_params["rmin"] + i*exact_params["dx"] for i in range(len(PSI_ADI[0,:,0]))]
    
nsteps_freq = TS.shape[0]
    
if method_indx in [3,4,5,6]: #SH-based
    tdE = np.zeros((nsteps_freq, ntraj))
    for istep in range(nsteps_freq):
        for itraj in range(ntraj):
            q.set(0, 0, QS[istep, itraj])
            E = get_Eadi(nst, q)
            
            tdE[istep, itraj] = E[STATES[istep, itraj]]
        
else: #Ehrenfest-based
    tdE = np.zeros((nsteps_freq, ntraj))
    for istep in range(nsteps_freq):
        for itraj in range(ntraj):
            q.set(0, 0, QS[istep, itraj])
            E = get_Eadi(nst, q)
            
            for ist in range(nst):
                tdE[istep, itraj] += E[ist]*np.linalg.norm(C_ADI[istep, itraj, ist])**2
                
for istep in range(nsteps_freq):
    print(F"{QS[istep,0]}  {tdE[istep,0]}")

In [ ]:
def extract_qd(prefix, _n, _states, _freq, _xmin, _xmax):
    fn = F"{prefix}/wfcr_snap_{_n*_freq}_dens_rep_1"
    which_cols = [0]
    nstates = len(_states)

    for state in _states:
        which_cols.append(1+state)
    data_exact = data_read.get_data_from_file2(fn, which_cols)

    ngrids = len(data_exact[0])

    #find necessaru region
    grids = np.array(data_exact[0])

    mask = np.logical_and(grids>_xmin, grids<_xmax)
    grids = grids[mask]

    # Get wfc array
    wfcs = []
    for ist in range(nstates):
        wfcs.append(np.array(data_exact[1+ist]))
    
    for ist in range(nstates):
        wfcs[ist] = wfcs[ist][mask]

    wfcs = np.array(wfcs)
    wfcs_tot = np.sum(wfcs, axis=0)
    
    tdE_exact = np.zeros(grids.shape)

    #Egrids = np.zeros((nstates, grids.shape[0]))
    for i, ix in enumerate(grids):
        q.set(0, 0, ix)
        E = get_Eadi(nstates, q)
    
        #nucl density
        chi2 = 0.
        for ist in range(nstates):
            chi2 += wfcs[ist,i]
        
        #norm = 0.
        #for ist in range(nstates):
        #    norm +=  wfcs[ist,i]/chi2
        #print(norm)
        #print(f"{E[0]}  {E[1]}  {E[2]}")
        for ist in range(nstates):
            #Egrids[ist, i] = np.real(E[ist])
            tdE_exact[i] += wfcs[ist,i]*np.real(E[ist])/chi2
            #tdE_exact[i] += (1/3)*np.real(E[ist])
    
    return grids, wfcs, wfcs_tot, tdE_exact

## Snapshot plots
Holstein

In [ ]:
## Snapshot plots
fig, axs = plt.subplots(2, 3, figsize=(15,6), gridspec_kw={"hspace": 0, "wspace": 0}, dpi=600)

################
#   COLUMN 0   #
################
# 0,0 TDPEC
n=1
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc1", n, [0,1], freq, xmin, xmax)

axs[0][0].set_xlim((-6, 6))
axs[0][0].set_ylim((-0.005,0.04))
axs[0][0].set_ylabel('Energy (Ha)', fontsize=25)
axs[0][0].tick_params(axis='both', labelsize=25)

axs[0][0].set_xticks([])

## TD PEC
axs[0][0].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][0].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][0].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][0].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,0 Nuclear distribution
axs[1][0].set_xlim((-6, 6))
axs[1][0].set_ylim([-0.03,1.03])
axs[1][0].set_xlabel(r'$R$ (Bohr)', fontsize=25)
axs[1][0].set_ylabel('Density', fontsize=25)

axs[1][0].set_xticks([-4, 0, 4])
axs[1][0].tick_params(axis='both', labelsize=25)

axs[1][0].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][0].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][0].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")

################
#   COLUMN 1   #
################
# 0,1 TDPEC
n=5
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc1", n, [0,1], freq, xmin, xmax)

axs[0][1].set_xlim((-6, 6))
axs[0][1].set_ylim((-0.005,0.04))

axs[0][1].set_xticks([])
axs[0][1].set_yticks([])

## TD PEC
axs[0][1].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][1].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][1].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][1].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][1].set_xlim((-6, 6))
axs[1][1].set_ylim([-0.03,1.03])
axs[1][1].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][1].set_xticks([-4, 0, 4])
axs[1][1].set_yticks([])
axs[1][1].tick_params(axis='both', labelsize=25)

axs[1][1].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][1].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][1].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")

################
#   COLUMN 2   #
################
# 0,1 TDPEC
n=7
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc1", n, [0,1], freq, xmin, xmax)

axs[0][2].set_xlim((-6, 6))
axs[0][2].set_ylim((-0.005,0.04))

axs[0][2].set_xticks([])
axs[0][2].set_yticks([])

## TD PEC
axs[0][2].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][2].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][2].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][2].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][2].set_xlim((-6, 6))
axs[1][2].set_ylim([-0.03,1.03])
axs[1][2].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][2].set_xticks([-4, 0, 4])
axs[1][2].set_yticks([])
axs[1][2].tick_params(axis='both', labelsize=25)

axs[1][2].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][2].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][2].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")
    
fig.align_labels()

In [ ]:
## Snapshot plots
fig, axs = plt.subplots(2, 3, figsize=(15,6), gridspec_kw={"hspace": 0, "wspace": 0}, dpi=600)

################
#   COLUMN 0   #
################
# 0,0 TDPEC
n=9
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc1", n, [0,1], freq, xmin, xmax)

axs[0][0].set_xlim((-6, 6))
axs[0][0].set_ylim((-0.005,0.04))
axs[0][0].set_ylabel('Energy (Ha)', fontsize=25)
axs[0][0].tick_params(axis='both', labelsize=25)

axs[0][0].set_xticks([])

## TD PEC
axs[0][0].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][0].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][0].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][0].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,0 Nuclear distribution
axs[1][0].set_xlim((-6, 6))
axs[1][0].set_ylim([-0.03,1.03])
axs[1][0].set_xlabel(r'$R$ (Bohr)', fontsize=25)
axs[1][0].set_ylabel('Density', fontsize=25)

axs[1][0].set_xticks([-4, 0, 4])
axs[1][0].tick_params(axis='both', labelsize=25)

axs[1][0].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][0].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][0].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")

################
#   COLUMN 1   #
################
# 0,1 TDPEC
n=12
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc1", n, [0,1], freq, xmin, xmax)

axs[0][1].set_xlim((-6, 6))
axs[0][1].set_ylim((-0.005,0.04))

axs[0][1].set_xticks([])
axs[0][1].set_yticks([])

## TD PEC
axs[0][1].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][1].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][1].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][1].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][1].set_xlim((-6, 6))
axs[1][1].set_ylim([-0.03,1.03])
axs[1][1].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][1].set_xticks([-4, 0, 4])
axs[1][1].set_yticks([])
axs[1][1].tick_params(axis='both', labelsize=25)

axs[1][1].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][1].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][1].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")

################
#   COLUMN 2   #
################
# 0,1 TDPEC
n=15
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc1", n, [0,1], freq, xmin, xmax)

axs[0][2].set_xlim((-6, 6))
axs[0][2].set_ylim((-0.005,0.04))

axs[0][2].set_xticks([])
axs[0][2].set_yticks([])

## TD PEC
axs[0][2].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][2].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][2].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][2].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][2].set_xlim((-6, 6))
axs[1][2].set_ylim([-0.03,1.03])
axs[1][2].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][2].set_xticks([-4, 0, 4])
axs[1][2].set_yticks([])
axs[1][2].tick_params(axis='both', labelsize=25)

axs[1][2].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][2].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][2].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")
    
fig.align_labels()

## Esch-Levine

In [ ]:
## Snapshot plots
fig, axs = plt.subplots(2, 3, figsize=(15,6), gridspec_kw={"hspace": 0, "wspace": 0}, dpi=600)

################
#   COLUMN 0   #
################
# 0,0 TDPEC
n=1
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc7", n, [0,1,2], freq, xmin, xmax)

axs[0][0].set_xlim((-3, 8))
axs[0][0].set_ylim((ymin, ymax))
axs[0][0].set_ylabel('Energy (Ha)', fontsize=25)
axs[0][0].tick_params(axis='both', labelsize=25)

axs[0][0].set_xticks([])

## TD PEC
axs[0][0].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][0].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][0].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][0].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,0 Nuclear distribution
axs[1][0].set_xlim((-3, 8))
axs[1][0].set_ylim([-0.025,0.875])
axs[1][0].set_xlabel(r'$R$ (Bohr)', fontsize=25)
axs[1][0].set_ylabel('Density', fontsize=25)

axs[1][0].set_xticks([-2, 7])
axs[1][0].tick_params(axis='both', labelsize=25)

axs[1][0].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][0].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][0].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")

################
#   COLUMN 1   #
################
# 0,1 TDPEC
n=2
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc7", n, [0,1,2], freq, xmin, xmax)

axs[0][1].set_xlim((-3, 8))
axs[0][1].set_ylim((ymin, ymax))

axs[0][1].set_xticks([])
axs[0][1].set_yticks([])

## TD PEC
axs[0][1].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][1].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][1].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][1].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][1].set_xlim((-3, 8))
axs[1][1].set_ylim([-0.025,0.875])
axs[1][1].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][1].set_xticks([-2, 7])
axs[1][1].set_yticks([])
axs[1][1].tick_params(axis='both', labelsize=25)

axs[1][1].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][1].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][1].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")

################
#   COLUMN 2   #
################
# 0,1 TDPEC
n=4
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc7", n, [0,1,2], freq, xmin, xmax)

axs[0][2].set_xlim((-3, xmax))
axs[0][2].set_ylim((ymin, ymax))

axs[0][2].set_xticks([])
axs[0][2].set_yticks([])

## TD PEC
axs[0][2].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][2].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][2].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][2].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][2].set_xlim((-3, xmax))
axs[1][2].set_ylim([-0.025,0.875])
axs[1][2].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][2].set_xticks([-2, 15])
axs[1][2].set_yticks([])
axs[1][2].tick_params(axis='both', labelsize=25)

axs[1][2].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][2].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][2].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")
    
fig.align_labels()

In [ ]:
nr_mark = 20
freq_mark = int(len(grids)/nr_mark)

## Snapshot plots
fig, axs = plt.subplots(2, 3, figsize=(15,6), gridspec_kw={"hspace": 0, "wspace": 0})

################
#   COLUMN 0   #
################
# 0,0 TDPEC
n=10
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc7", n, [0,1,2], freq, xmin, xmax)

axs[0][0].set_xlim((xmin, xmax))
axs[0][0].set_ylim((ymin, ymax))
axs[0][0].set_ylabel('Energy (Ha)', fontsize=25)
axs[0][0].tick_params(axis='both', labelsize=25)

axs[0][0].set_xticks([])

## TD PEC
axs[0][0].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][0].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][0].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][0].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,0 Nuclear distribution
axs[1][0].set_xlim((xmin, xmax))
axs[1][0].set_ylim([-0.01,0.36])
axs[1][0].set_xlabel(r'$R$ (Bohr)', fontsize=25)
axs[1][0].set_ylabel('Density', fontsize=25)

axs[1][0].set_xticks([-15, 0, 15])
axs[1][0].tick_params(axis='both', labelsize=25)

axs[1][0].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][0].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    #axs[1][0].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=1, marker="o", markevery=[x for x in range(0, len(grids), freq_mark)] )
    axs[1][0].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")

################
#   COLUMN 1   #
################
# 0,1 TDPEC
n=15
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc7", n, [0,1,2], freq, xmin, xmax)

axs[0][1].set_xlim((xmin, xmax))
axs[0][1].set_ylim((ymin, ymax))

axs[0][1].set_xticks([])
axs[0][1].set_yticks([])

## TD PEC
axs[0][1].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][1].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][1].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][1].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][1].set_xlim((xmin, xmax))
axs[1][1].set_ylim([-0.01,0.36])
axs[1][1].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][1].set_xticks([-15, 0, 15])
axs[1][1].set_yticks([])
axs[1][1].tick_params(axis='both', labelsize=25)

axs[1][1].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][1].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][1].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")
    #axs[1][1].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=1, marker="o", markevery=[x for x in range(0, len(grids), freq_mark)] )

################
#   COLUMN 2   #
################
# 0,1 TDPEC
n=20
grids, wfcs, wfcs_tot, tdE_exact = extract_qd("wfc7", n, [0,1,2], freq, xmin, xmax)

axs[0][2].set_xlim((xmin, xmax))
axs[0][2].set_ylim((ymin, ymax))

axs[0][2].set_xticks([])
axs[0][2].set_yticks([])

## TD PEC
axs[0][2].set_title(r"$t=$"+F'{TS[n]/41:.1f} fs', fontsize=25)

axs[0][2].plot(grids, tdE_exact, linewidth=5, color = "black", zorder=-10)

for ist in range(nst):
    axs[0][2].plot(xs, Es[:, ist], label=F"E{ist}", linewidth=2, color = colors[clrs_index[ist]], zorder=0)

for traj in range(ntraj):
    axs[0][2].scatter(QS[n, traj],tdE[n, traj], s=60, facecolors='none', edgecolors=colors["61"], zorder=-11)

# 1,1 Nuclear distribution
axs[1][2].set_xlim((xmin, xmax))
axs[1][2].set_ylim([-0.01,0.36])
axs[1][2].set_xlabel(r'$R$ (Bohr)', fontsize=25)

axs[1][2].set_xticks([-15, 0, 15])
axs[1][2].set_yticks([])
axs[1][2].tick_params(axis='both', labelsize=25)

axs[1][2].hist(QS[n, :], [xmin + 10*dx*i for i in range(int(nx/10))], color=colors["61"], density=True)

axs[1][2].plot(grids, wfcs_tot, color="black", linewidth=5)
for ist in range(nst):
    axs[1][2].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=2, linestyle="dashed")
    #axs[1][2].plot(grids, wfcs[ist], color=colors[clrs_index[ist]], linewidth=1, marker="o", markevery=[x for x in range(0, len(grids), freq_mark)] )
    
fig.align_labels()